# Advanced Data Structures and Storage
## Session 20 (Exercises on Column-Family Databases)

*Submit your solutions in the Blackboard. I will ask some of you to discuss your solutions in class next session.*

## Part 1

Fire up a *Cassandra Query Language* (CQL) session as follows:

- Go to [DataStax](https://www.datastax.com/dev) and select "Try free"
- Login (and fill the form, if requested)
- Select `Create Database`
- As database name and keyspace name, use `twitter`
- Choose your favorite provider and your closest region
- Wait a minute until the database is up and running
- Select the tab `CQL Console` above
- Activate the keyspace with the USE command (see session 19's slides).

**OR**

- Head over to [Github Codespaces](github.com/codespaces)
- Run `docker pull cassandra` and `docker run -d cassandra`
  - If you re-open the codespace after a while, you may need to run `docker ps -a` and `docker start <container_name>` to start the container again
- Run `pip install cassandra-driver`
- Make sure the Cassandra demo notebook is running

In [1]:
import cassandra

## Part 2

We want to model a database that stores [X (formerly Twitter)](https://twitter.com) users and the tweets they post. So far, there are two users and two tweets. In SQL, they would be stored using these tables:

*USERS table:*

| Author handle *(primary key)* 	| Author name 	| E-mail         	|
|----------------------	|----------------	|----------------	|
| @john                	| John Doe       	| john@gmail.com 	|
| @jane                	| Jane Doe       	| jane@gmail.com 	|

*TWEETS table:*

| Tweet ID *(primary key)* 	| Author handle 	| Body                   	|
|------------------------	|---------------	|------------------------	|
| 19000                  	| @jane         	| "My first tweet"       	|
| 34500                  	| @jane         	| "I got the hang of it" 	|

*If we want to display one tweet, how many tables do we need to traverse?* Justify your answer.

Hint: in case you're not familiar with Twitter, have a look at it to make sure you **understand what information is shown in a tweet**.

### An usual tweet usually is composed of 3 things: The body, the author handle and the Author name. Thus, to be able to show one tweet we would have to traverse 2 tables. Th tweets table for the author handle and body and then the users table to get the author name.

## Part 3

*How would you model that small database using Cassandra*? Hint: use two tables. Follow the principle that "columns that are *used* together should be put together". Using your model, *how many tables do we need to traverse to display one tweet?*

You can put a screenshot of an Excel spreadsheet (bonus: [write the tables in Markdown format](https://www.tablesgenerator.com/markdown_tables)).

### users Table

| author_handle | author_name | email           |
|---------------|-------------|-----------------|
| @john         | John Doe    | john@gmail.com  |
| @jane         | Jane Doe    | jane@gmail.com  |

### tweets Table

| tweet_id | author_handle | author_name | tweet_body        |
|----------|---------------|-------------|-------------------|
| 19000    | @jane         | Jane Doe    | My first tweet    |
| 34500    | @jane         | Jane Doe    | I got the hang of it |


## Part 4

Create your two tables in Cassandra and fill them up with all available data records from Part 2. Paste your CQL code here.

In [20]:
from cassandra.cluster import Cluster

cluster = Cluster(['172.17.0.2'])
session = cluster.connect()

session.execute("""
CREATE KEYSPACE IF NOT EXISTS adss
WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
""")

session.set_keyspace('adss')

In [21]:
#Lets start creating the tables

session.execute(
"""
CREATE TABLE IF NOT EXISTS tweets (tweet_id int PRIMARY KEY, author_handle text , author_name text, tweet_body text);
"""
)

session.execute(
"""
CREATE TABLE IF NOT EXISTS users (author_handle text PRIMARY KEY, author_name text, email text);
"""
)

In [22]:
#Add the rows to the tables
result = session.execute(
"""
INSERT INTO tweets (tweet_id, author_handle, author_name, tweet_body) VALUES (19000, '@jane', 'Jane Doe', 'My first tweet');
"""
)

result = session.execute(
"""
INSERT INTO tweets (tweet_id, author_handle, author_name, tweet_body) VALUES (34500, '@jane', 'Jane Doe', 'I got the hang of it');
"""
)

result = session.execute(
"""
INSERT INTO users (author_handle, author_name, email) VALUES ('@john', 'John Doe', 'john@gmail.com');
"""
)

result = session.execute(
"""
INSERT INTO users (author_handle, author_name, email) VALUES ('@jane', 'Jane Doe', 'jane@gmail.com');
"""
)



## Part 5

Use CQL to print the contents of both tables.

In [23]:
#Check tweets table
result = session.execute(
"""
SELECT * FROM tweets;
"""
)
print(list(result))

[Row(tweet_id=19000, author_handle='@jane', author_name='Jane Doe', tweet_body='My first tweet'), Row(tweet_id=34500, author_handle='@jane', author_name='Jane Doe', tweet_body='I got the hang of it')]


In [24]:
#Check users table
result = session.execute(
"""
SELECT * FROM users;
"""
)
print(list(result))

[Row(author_handle='@jane', author_name='Jane Doe', email='jane@gmail.com'), Row(author_handle='@john', author_name='John Doe', email='john@gmail.com')]


## Part 6

Write CQL code that retrieves Jane's tweets.

Hint: see the second to last slide from Session 19 ("Efficient Data Selection").

Bonus: make it work without filtering.

In [31]:
result = session.execute(
    '''
    SELECT * FROM tweets WHERE author_handle = '@jane' ALLOW FILTERING
    '''
)

print(list(result))

[Row(tweet_id=19000, author_handle='@jane', author_name='Jane Doe', tweet_body='My first tweet'), Row(tweet_id=34500, author_handle='@jane', author_name='Jane Doe', tweet_body='I got the hang of it')]


In [32]:
session.execute(
    '''
    CREATE INDEX IF NOT EXISTS handle ON tweets (author_handle)
    '''
)

In [33]:
result = session.execute(
    '''
    SELECT * FROM tweets WHERE author_handle = '@jane'
    '''
)

print(list(result))

[Row(tweet_id=19000, author_handle='@jane', author_name='Jane Doe', tweet_body='My first tweet'), Row(tweet_id=34500, author_handle='@jane', author_name='Jane Doe', tweet_body='I got the hang of it')]


## Part 7

Denormalize the two tables from Part 2 into a single, wider table with five rows. Use `n/a` where necessary.

Hint: see the slide *"Cassandra Denormalization Example"* from Session 19.

| author_handle | author_name |  email         | tweet_id| tweet_body        |
|---------------|-------------|----------------|---------|-------------------|
| @jane         | Jane Doe    | jane@gmail.com |19000    | My first tweet    |
| @jane         | Jane Doe    | jane@gmail.com |34500    | I got the hang of it |
| @john         | John Doe    | john@gmail.com | n/a     | n/a               | 